In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from datetime import datetime
from IPython.display import display, Markdown
from pathvalidate import sanitize_filename

import utils

from prompts import Job_Post, Resume_Builder

In [4]:
## Inputs
my_files_dir = "my_applications"  # location for all job and resume files
job_file = "job.txt"  # filename with job post text. The entire job post can be pasted in this file, as is.
raw_resume_file = "resume_raw.yaml"  # filename for raw resume yaml. See example in repo for instructions.

# Model for matching resume to job post (except extraction chains)
# gpt-4 is not publicly available yet and can be 20-30 times costlier than the default model gpt-3.5-turbo.
# Simple extraction chains are hardcoded to use the cheaper gpt-3.5 model.
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

# temperature lower than 1 is preferred. temperature=0 returns deterministic output
temperature = 0.5

llm_kwargs = dict(
    model_name=openai_model_name,
    model_kwargs=dict(top_p=0.6, frequency_penalty=0.1),
)

In [5]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
)
parsed_job = job_post.parse_job_post(verbose=False)

company_name = parsed_job["company"]
job_title = parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
job_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
print(f"#filename: {job_filename}.job\n")
utils.write_yaml(parsed_job, filename=f"{job_filename}.job")

#filename: my_applications\20230915__NOIR__VueJS Developer.job

company: NOIR
job_title: VueJS Developer
team:
job_summary: VueJS Developer position at a global record label in London, working
  on critical projects for their new online music shopping channel. The company offers
  flexible working hours, home working, and amazing offices.
salary: Â£80,000 - Â£100,000 per annum
duties:
- Developing web applications using VueJS, HTML5, CSS3, TypeScript, GIT Version Control,
  JavaScript, SASS, jQuery, React, and AJAX
- Working on projects critical to the launch of the company's new online music shopping
  channel
- Collaborating with cross-functional teams to deliver high-quality software solutions
qualifications:
- Experience in VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS,
  jQuery, React, and AJAX
- Ability to learn and adapt to new technologies
- Strong problem-solving and debugging skills
- Excellent communication and teamwork skills
is_fully_remote: true
te

In [6]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    resume=utils.read_yaml(filename=os.path.join(my_files_dir, raw_resume_file)),
    parsed_job=parsed_job,
    llm_kwargs=llm_kwargs,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [7]:
# Step 3 - Rephrase unedited experiences
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
utils.write_yaml(dict(experiences=experiences))

experiences:
- company: SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY CO., LTD. Shanghai, China
  titles:
  - name: Web Front-end Development Engineer - Software R&D Department
    startdate: 2021.07
    enddate: 2022.08
    # highlights:
    # the unedited tag in experiences can be used to automatically have the LLM rewrite it as highlights
  highlights:
  - Developed web-based features using VueJS and Typescript for customized PDF and
    Excel reports with 200,000+ unique users per month, demonstrating proficiency
    in VueJS, Typescript, and web application development.
  - Full-stack developed a material calculation system using VueJS and Flask, resulting
    in a significant reduction in production time for users, highlighting strong problem-solving
    and teamwork skills.
  - Developed an automatic 3D house layout model generation system in a pre-research
    using VueJS and BabylonJS, showcasing problem-solving skills and ability to work
    on critical projects.
- company: TUQI 

In [8]:
# Step 4 - Rephrase projects
my_resume.experiences = experiences

projects = my_resume.rewrite_projects_desc(verbose=False)
utils.write_yaml(dict(projects=projects))

projects:
- title: Portfolio
  link:
  - demoUrl: https://www.chenmo1212.cn?f=cv
    demoIcon: global
    githubUrl: https://github.com/Chenmo1212/homePage
    githubIcon: github
  skills: TweenLiteJS, ScrollMagicJS, Python, FLask,  REST API, pytest, Github Action
  highlights:
  - Developed a personal portfolio using native Web front-end technology and Python
    back-end, showcasing proficiency in web development and backend programming.
  - Developed a REST API using Python and Flask, enabling seamless communication between
    front-end and back-end systems.
  - Utilized TweenLiteJS and ScrollMagicJS for web animations, enhancing user experience
    on web pages.
  - Implemented automated testing using pytest and Github Action, ensuring software
    quality and reliability.
- title: Sonder Notebook
  link:
  - demoUrl: https://note.chenmo1212.cn?f=cv
    demoIcon: global
    githubUrl: ''
    githubIcon: github
  skills: Shell, Python, Git, Docsify, Github Action
  highlights:
  - 

In [9]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits2 = utils.read_yaml(edits)
    if "experiences" in edits2:
        experiences = edits2["experiences"]
    if "projects" in edits2:
        projects = edits2["projects"]
    if "skills" in edits2:
        skills = edits2["skills"]
    if "summary" in edits2:
        summary = edits2["summary"]

In [10]:
# Step 5 - Extract skills
# This will match the required skills from the job post with your resume sections
# Outputs a combined list of skills extracted from the job post and included in the raw resume
my_resume.experiences = experiences
my_resume.projects = projects

skills = my_resume.extract_matched_skills(verbose=False)
utils.write_yaml(dict(skills=skills))

skills:
- category: Technical
  skills:
  - VueJS
  - TypeScript
  - React
  - ReactJS
  - Npm
  - Webpack
  - Node.js
  - Git
  - Python
  - Flask
  - Devops with Github workflow
  - C++
  - openGL
  - Blender
  - Unity
  - Unreal Engine 5
  - Photoshop
  - Premiere
  - Adobe XD
  - Figma
- category: Non-technical
  skills:
  - Problem-solving
  - Teamwork
  - Communication
  - Memory and Recall
  - Autonomy
  - Written Communication
  - Verbal Communication
  - Time Management
  - Collaboration
  - Diversity and Inclusion


In [11]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits3 = utils.read_yaml(edits)
    if "experiences" in edits3:
        experiences = edits3["experiences"]
    if "projects" in edits3:
        projects = edits3["projects"]
    if "skills" in edits3:
        skills = edits3["skills"]
    if "summary" in edits3:
        summary = edits3["summary"]

In [12]:
# Step 6 - Create a resume summary
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

summary = my_resume.write_summary(
    verbose=True,
)
utils.write_yaml(dict(summary=summary))



> Entering new  chain...
Prompt after formatting:
System: You are an expert technical writer. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
NOIR
VueJS Developer position at a global record label in London, working on critical projects for their new online music shopping channel. The company offers flexible working hours, home working, and amazing offices.
Human: <Resume>
Education:
- Master's in Computer Science - Augment and Virtual Reality
- Bachelor's in Information management and information system

Experience:
- 

- 1 years experience in:
  - Developed web-based features using VueJS and Typescript for customized PDF and Excel reports with 200,000+ unique users per month, demonstrating proficiency in VueJS, Typescript, and web application development.
  - Full-stack developed a material calculation system using VueJS and Flask, resulting in a significant reduction in production time for users, highlighting strong 

In [13]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed.
edits = """ """

edits = edits.strip()
if edits:
    edits4 = utils.read_yaml(edits)
    if "experiences" in edits4:
        experiences = edits4["experiences"]
    if "projects" in edits4:
        projects = edits4["projects"]
    if "skills" in edits4:
        skills = edits4["skills"]
    if "summary" in edits4:
        summary = edits4["summary"]

In [14]:
# Step 7 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

#filename: my_applications\20230915__NOIR__VueJS Developer.yaml

basic:
  name: Long Pan
  address:
  - Dublin, Ireland
  contact:
    email: panlo@tcd.ie
    phone: (+353) 087-4615379
  websites:
    # optionally add icons to websites from fontawesome - http://mirrors.ctan.org/fonts/fontawesome/doc/fontawesome.pdf
  - text: chenmo1212.cn
    url: https://www.chenmo1212.cn?f=cv
    icon: global
  - text: Long Pan
    url: https://www.linkedin.com/in/long-pan-chandler/
    icon: linkedin
  - text: Chenmo1212
    url: https://github.com/Chenmo1212?f=cv
    icon: github


# summary will be overwritten by the LLM. Leave blank.
summary: Highly skilled VueJS Developer with a Master's in Computer Science and 1
  year of experience in developing web-based features using VueJS and Typescript.
  Proficient in VueJS, Typescript, and web application development, I have successfully
  handled projects with 200,000+ unique users per month. Additionally, I have demonstrated
  strong problem-solving a

In [16]:
# Step 8 - Identify resume improvements
# A previously generated resume can also be used here by manually providing resume_filename. Requires the associated parsed job file.
final_resume = Resume_Builder(
    resume=utils.read_yaml(filename=f"{resume_filename}.yaml"),
    parsed_job=utils.read_yaml(filename=f"{resume_filename}.job"),
    is_final=True,
    llm_kwargs=llm_kwargs,
)
improvements = final_resume.suggest_improvements(verbose=True)
utils.write_yaml(dict(improvements=improvements))



> Entering new  chain...
Prompt after formatting:
System: You are an expert critic. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
The ideal candidate is able to perform the following duties:
- Developing web applications using VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS, jQuery, React, and AJAX.
- Collaborating with the team to ensure the successful launch of the online music shopping channel.
- Participating in code reviews and providing feedback to improve the quality of the codebase.
- Keeping up to date with the latest web development trends and technologies.

The ideal candidate has the following qualifications:
- Experience with VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS, jQuery, React, and AJAX.
- Strong problem-solving skills and attention to detail.
- Ability to work independently and as part of a team.
- Excellent communication and collaboration skills.
- 


> Finished chain.
Chain output:
<Plan>
To follow the <Instruction> and meet all the <Criteria>, I will:
1. Review the <Resume> for spelling and grammar errors.
2. Identify areas in the <Resume> that can be improved to better showcase the candidate's qualifications and meet the requirements of the <Job Posting>.
3. Provide specific suggestions for improvement in each section of the <Resume>.
4. Compile all suggestions into the <Final Answer> and include a section for Spelling and Grammar.

<Additional Steps>
None.

<Work>
Spelling and Grammar:
- In the first experience highlight, change "proficiency" to "proficient" to match the tense used in the sentence.
- In the second experience highlight, change "highlighting" to "highlighted" to match the tense used in the sentence.
- In the third experience highlight, change "showcasing" to "showcased" to match the tense used in the sentence.
- In the second project highlight, change "resulting in improved accessibility and user experience" to "

In [17]:
# Step 9 - Generate pdf from yaml
# Most common errors during pdf generation occur due to special characters. Escape them with backslashes in the yaml, e.g. $ -> \$
pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
# display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))

PDF could not be generated. See latexmk logs my_applications\latexmk_temp.log


ValueError: PDF could not be generated. See latexmk logs my_applications\latexmk_temp.log

In [79]:
utils.generate_new_tex(yaml_file=f"{resume_filename}.yaml")

Done


In [23]:
yaml_file=f"{resume_filename}.yaml"
template_file = "resume.tex"

dirname, basename = os.path.split(yaml_file)
filename, ext = os.path.splitext(basename)
filename = os.path.join(dirname, filename)


In [24]:
template = env.get_template(template_file)

yaml_data = read_yaml(filename=yaml_file)
latex_string = template.render(**yaml_data)

with open(f"{filename}.tex", "wt") as stream:
    stream.write(latex_string)

NameError: name 'env' is not defined